In [1]:
from pyiron_workflow import Workflow, as_function_node, as_macro_node, as_dataclass_node
from pyiron_workflow.node import Node
from pyiron_workflow.nodes.function import Function
from pyiron_workflow.nodes.composite import Composite
from dataclasses import dataclass, is_dataclass, fields, field
from semantikon.typing import u
from semantikon.converter import semantikon_class, parse_input_args, parse_output_args, parse_metadata, meta_to_dict
from owlready2 import *
from pprint import pprint
from rdflib import Graph, Namespace, URIRef, BNode
from rdflib import Literal as rdfLit
import graphviz
from rdflib.namespace import RDF, RDFS
import random
from bfo import visualize
import numpy as np
from typing import Optional, Callable
from pydantic import BaseModel, Field

In [2]:
onto_path.append("/mnt/e/pyiron/pmdco_interface/main/pyiron_interface_pmdco/core-ontology-main_20250623/core-ontology-main")
pmdco = get_ontology("/mnt/e/pyiron/pmdco_interface/main/pyiron_interface_pmdco/core-ontology-main_20250623/core-ontology-main/pmdco.owl").load()

base_iris = set()
for cls in pmdco.classes():
    iri = cls.iri
    # Trim to last slash or hash
    if "#" in iri:
        base = iri[:iri.rfind("#")+1]
    else:
        base = iri[:iri.rfind("/")+1]
    base_iris.add(base)
    
for base in base_iris:
    print(base)

obo = pmdco.get_namespace("http://purl.obolibrary.org/obo/")
pmdco = pmdco.get_namespace("https://w3id.org/pmd/co/")

http://purl.obolibrary.org/obo/
https://w3id.org/pmd/co/


In [3]:
@semantikon_class
@dataclass
class AtomisticSampleSimulationCell:
    simulation_cell_lengths: u(str, units="ANGSTROM", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    simulation_cell_vectors: u(str, units="ANGSTROM", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    simulation_cell_angles: u(str, units="DEGREES", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    simulation_cell_volume: u(float, units="ANGSTROM3", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None

@semantikon_class
@dataclass
class AtomisticSampleChemicalSpecies: 
    atoms: u(Callable, bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None

@semantikon_class
@dataclass
class AtomisticSampleUnitCell:
    lattice_parameter_a: u(float, units="ANGSTROM", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    lattice_parameter_b: u(float, units="ANGSTROM", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    lattice_parameter_c: u(float, units="ANGSTROM", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    lattice_angle_alpha: u(float, units="DEGREES", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    lattice_angle_beta: u(float, units="DEGREES", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    lattice_angle_gamma: u(float, units="DEGREES", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    lattice_volume: u(float, units="ANGSTROM3", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None

@semantikon_class
@dataclass
class AtomisticSampleSPGLibAnalysis:
    space_group: u(str, bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    bravais_lattice: u(str, bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    analysis_precision: u(float, units="ANGSTROM", bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None
    analysis_libraray_name: u(str, bfo_prop=obo["BFO_0000178"], bfo_type=obo["OBI_0001933"]) = None

@semantikon_class
@dataclass
class AtomisticSample():
    simulation_cell: u(AtomisticSampleSimulationCell, bfo_prop=obo["BFO_0000178"], bfo_type=obo["IAO_0000027"])
    chemical_species: u(AtomisticSampleChemicalSpecies, bfo_prop=obo["BFO_0000178"], bfo_type=obo["IAO_0000027"])
    unit_cell: u(AtomisticSampleUnitCell, bfo_prop=obo["BFO_0000178"], bfo_type=obo["IAO_0000027"])
    crystal_defect: u(list, bfo_prop=obo["BFO_0000178"], bfo_type=obo["IAO_0000027"]) = field(default_factory=lambda: []) 
    analysis: u(list, bfo_prop=obo["BFO_0000178"], bfo_type=obo["IAO_0000027"]) = field(default_factory=lambda: [])

In [4]:
class SimulationCellComponent(BaseModel):
    value: str | float | None = None
    bfo_prop: str
    bfo_type: str

class PydanticAtomisticSampleSimulationCell(BaseModel):
    simulation_cell_lengths: SimulationCellComponent
    simulation_cell_vectors: SimulationCellComponent
    simulation_cell_angles: SimulationCellComponent
    simulation_cell_volume: SimulationCellComponent

cell = PydanticAtomisticSampleSimulationCell(
    simulation_cell_lengths={
        "bfo_prop": "BFO_0000178",
        "bfo_type": "OBI_0001933",
    },
    simulation_cell_vectors={
        "bfo_prop": "BFO_0000178",
        "bfo_type": "OBI_0001933",
    },
    simulation_cell_angles={
        "bfo_prop": "BFO_0000178",
        "bfo_type": "OBI_0001933",
    },
    simulation_cell_volume={
        "bfo_prop": "BFO_0000178",
        "bfo_type": "OBI_0001933",
    },
)

In [5]:
cell.model_dump()

{'simulation_cell_lengths': {'value': None,
  'bfo_prop': 'BFO_0000178',
  'bfo_type': 'OBI_0001933'},
 'simulation_cell_vectors': {'value': None,
  'bfo_prop': 'BFO_0000178',
  'bfo_type': 'OBI_0001933'},
 'simulation_cell_angles': {'value': None,
  'bfo_prop': 'BFO_0000178',
  'bfo_type': 'OBI_0001933'},
 'simulation_cell_volume': {'value': None,
  'bfo_prop': 'BFO_0000178',
  'bfo_type': 'OBI_0001933'}}

In [6]:
class PydanticFieldsAtomisticSampleSimulationCell(BaseModel):
    simulation_cell_lengths: str | None = Field(
        None, bfo_prop="BFO_0000178", bfo_type="OBI_0001933"
    )
    simulation_cell_vectors: str | None = Field(
        None, bfo_prop="BFO_0000178", bfo_type="OBI_0001933"
    )
    simulation_cell_angles: str | None = Field(
        None, bfo_prop="BFO_0000178", bfo_type="OBI_0001933"
    )
    simulation_cell_volume: float | None = Field(
        None, bfo_prop="BFO_0000178", bfo_type="OBI_0001933"
    )

In [7]:
cell_1 = PydanticFieldsAtomisticSampleSimulationCell(simulation_cell_lengths="[1.0,0.0,0.0][0.0,1.0,0.0][0.0,0.0,1.0]")
cell_1.model_dump()

{'simulation_cell_lengths': '[1.0,0.0,0.0][0.0,1.0,0.0][0.0,0.0,1.0]',
 'simulation_cell_vectors': None,
 'simulation_cell_angles': None,
 'simulation_cell_volume': None}

In [8]:
cell_1.model_json_schema()

{'properties': {'simulation_cell_lengths': {'anyOf': [{'type': 'string'},
    {'type': 'null'}],
   'bfo_prop': 'BFO_0000178',
   'bfo_type': 'OBI_0001933',
   'default': None,
   'title': 'Simulation Cell Lengths'},
  'simulation_cell_vectors': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'bfo_prop': 'BFO_0000178',
   'bfo_type': 'OBI_0001933',
   'default': None,
   'title': 'Simulation Cell Vectors'},
  'simulation_cell_angles': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'bfo_prop': 'BFO_0000178',
   'bfo_type': 'OBI_0001933',
   'default': None,
   'title': 'Simulation Cell Angles'},
  'simulation_cell_volume': {'anyOf': [{'type': 'number'}, {'type': 'null'}],
   'bfo_prop': 'BFO_0000178',
   'bfo_type': 'OBI_0001933',
   'default': None,
   'title': 'Simulation Cell Volume'}},
 'title': 'PydanticFieldsAtomisticSampleSimulationCell',
 'type': 'object'}